# OpenAI Tools
These output parsers extract tool calls from OpenAI's function calling API responses. This means they are only usable with models that support function calling, and specifically the latest tools and tool_choice parameters. We recommend familiarizing yourself with [function calling](https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/#request-passing-tool-outputs-to-model) before reading this guide.

There are a few different variants of output parsers:

* JsonOutputToolsParser: Returns the arguments of the function call as JSON
* JsonOutputKeyToolsParser: Returns the value of specific key in the function call as JSON
* PydanticToolsParser: Returns the arguments of the function call as a Pydantic Model

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

In [2]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [3]:
model = ChatOpenAI(model="gpt-4-turbo", temperature=0).bind_tools([Joke])

In [4]:
model.kwargs["tools"]

[{'type': 'function',
  'function': {'name': 'Joke',
   'description': 'Joke to tell user.',
   'parameters': {'type': 'object',
    'properties': {'setup': {'description': 'question to set up a joke',
      'type': 'string'},
     'punchline': {'description': 'answer to resolve the joke',
      'type': 'string'}},
    'required': ['setup', 'punchline']}}}]

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful assistant"), ("user", "{input}")]
)

## JsonOutputToolsParser

In [6]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
parser = JsonOutputToolsParser()
chain = prompt | model | parser

In [7]:
chain.invoke({"input": "tell me a joke"})

[{'args': {'setup': "Why don't skeletons fight each other?",
   'punchline': "They don't have the guts."},
  'type': 'Joke'}]

## JsonOutputKeyToolsParser
This merely extracts a single key from the returned response. This is useful for when you are passing in a single tool and just want it's arguments.

In [8]:
from typing import List

from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser
parser = JsonOutputKeyToolsParser(key_name="Joke")
chain = prompt | model | parser

In [9]:

chain.invoke({"input": "tell me a joke"})

[{'setup': "Why don't skeletons fight each other?",
  'punchline': "They don't have the guts."}]

Certain models can return **multiple tool** invocations each call, so by default the output is a list. If we just want to return the first tool invocation, we can specify first_tool_only=True

In [10]:
parser = JsonOutputKeyToolsParser(key_name="Joke", first_tool_only=True)
chain = prompt | model | parser
chain.invoke({"input": "tell me a joke"})

{'setup': "Why don't skeletons fight each other?",
 'punchline': "They don't have the guts."}

## PydanticToolsParser
This builds on top of `JsonOutputToolsParser` but passes the results to a Pydantic Model. This allows for further validation should you choose.

In [22]:
from langchain.output_parsers.openai_tools import PydanticToolsParser

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


parser = PydanticToolsParser(tools=[Joke])

model = ChatOpenAI(model="gpt-4-turbo", temperature=0.9).bind_tools([Joke])
chain = prompt | model | parser

In [23]:
chain.invoke({"input": "tell me a joke"})

[Joke(setup='What do you get when you cross a snowman with a vampire?', punchline='Frostbite!')]